In [1]:
import os
print(os.getcwd())
os.chdir("..")
from speos.utils.config import Config
#from speos.pipeline import InferencePipeline
#from speos.explanation import MessagePassingExplainer
from speos.models import ModelBootstrapper
from speos.datasets import DatasetBootstrapper
from speos.preprocessing.mappers import GWASMapper, AdjacencyMapper

/home/icb/florin.ratajczak/ppi-core-genes/speos


In [3]:
import os
print(os.getcwd())
os.chdir("..")

/home/icb/florin.ratajczak/ppi-core-genes/speos


In [2]:
config = Config()
#config.parse_yaml("configs/config_cardiovascular_film.yaml")
config.parse_yaml("configs/config_cardiovascular_tag.yaml")

mappings = GWASMapper(config.input.gene_sets, config.input.gwas).get_mappings(
            config.input.tag, fields=config.input.field)

tag = "" if config.input.adjacency == "all" else config.input.adjacency
adjacencies = AdjacencyMapper(config.input.adjacency_mappings).get_mappings(tag, fields=config.input.adjacency_field)

dataset = DatasetBootstrapper(
            mappings, adjacencies, holdout_size=config.input.holdout_size, name=config.name, config=config).get_dataset()

node_data = dataset.data
input_dim = node_data.x.shape[1]

model = ModelBootstrapper(
            config, input_dim, len(adjacencies)).get_model()

cardiovascular_tag 2022-08-24 11:56:23,670 [INFO] speos.preprocessing.preprocessor: Using Adjacency matrices: ['IntActDirect']
cardiovascular_tag 2022-08-24 11:56:23,670 [INFO] speos.preprocessing.preprocessor: Using 8 mappings with ground truth ./data/mendelian_gene_sets/Cardiovascular_Disease_genes.bed 


In [3]:
dataset.data.edge_index[:,0]

tensor([   14, 13182])

In [3]:
config.name = "cardiovascular_tag_outer_1_fold_0"
import os

from speos.helpers import CheckPointer
checkpointer = CheckPointer(
                model, config.model.save_dir + config.name, mode=config.es.mode)
checkpointer.restore()

(7, 0.23570190641243746)

In [13]:
import torch
os.environ["CUDA_VISIBLE_DEVICES"]=""
torch.cuda.is_available()

True

In [5]:
import os.path as osp

import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

In [7]:
from inspect import signature
from math import sqrt
from typing import Optional

import torch
from torch import Tensor

from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import k_hop_subgraph, to_networkx


def set_masks(model: torch.nn.Module, mask: Tensor, edge_index: Tensor,
              apply_sigmoid: bool = True):
    """Apply mask to every graph layer in the model."""
    loop_mask = edge_index[0] != edge_index[1]

    # Loop over layers and set masks on MessagePassing layers:
    for module in model.modules():
        if isinstance(module, MessagePassing):
            module._explain = True
            module._edge_mask = mask
            module._loop_mask = loop_mask
            module._apply_sigmoid = apply_sigmoid


def clear_masks(model: torch.nn.Module):
    """Clear all masks from the model."""
    for module in model.modules():
        if isinstance(module, MessagePassing):
            module._explain = False
            module._edge_mask = None
            module._loop_mask = None
            module._apply_sigmoid = True
    return module


class CaptumModel(torch.nn.Module):
    def __init__(self, model: torch.nn.Module, mask_type: str = "edge",
                 output_idx: Optional[int] = None):
        super().__init__()
        assert mask_type in ['edge', 'node', 'node_and_edge']

        self.mask_type = mask_type
        self.model = model
        self.output_idx = output_idx

    def forward(self, mask, *args):
        """"""
        # The mask tensor, which comes from Captum's attribution methods,
        # contains the number of samples in dimension 0. Since we are
        # working with only one sample, we squeeze the tensors below.
        assert mask.shape[0] == 1, "Dimension 0 of input should be 1"
        if self.mask_type == "edge":
            assert len(args) >= 2, "Expects at least x and edge_index as args."
        if self.mask_type == "node":
            assert len(args) >= 1, "Expects at least edge_index as args."
        if self.mask_type == "node_and_edge":
            assert args[0].shape[0] == 1, "Dimension 0 of input should be 1"
            assert len(args[1:]) >= 1, "Expects at least edge_index as args."

        # Set edge mask:
        if self.mask_type == 'edge':
            set_masks(self.model, mask.squeeze(0), args[1],
                      apply_sigmoid=False)
        elif self.mask_type == 'node_and_edge':
            set_masks(self.model, args[0].squeeze(0), args[1],
                      apply_sigmoid=False)
            args = args[1:]

        if self.mask_type == 'edge':
            x = self.model(*args)

        elif self.mask_type == 'node':
            x = self.model(mask.squeeze(0), *args)

        else:
            x = self.model(mask[0], *args)

        # Clear mask:
        if self.mask_type in ['edge', 'node_and_edge']:
            clear_masks(self.model)

        if self.output_idx is not None:
            x = x[self.output_idx].unsqueeze(0)

        return x


def to_captum(model: torch.nn.Module, mask_type: str = "edge",
              output_idx: Optional[int] = None) -> torch.nn.Module:
    r"""Converts a model to a model that can be used for
    `Captum.ai <https://captum.ai/>`_ attribution methods.

    .. code-block:: python

        from captum.attr import IntegratedGradients
        from torch_geometric.nn import GCN, from_captum

        model = GCN(...)
        ...  # Train the model.

        # Explain predictions for node `10`:
        output_idx = 10

        captum_model = to_captum(model, mask_type="edge",
                                 output_idx=output_idx)
        edge_mask = torch.ones(num_edges, requires_grad=True, device=device)

        ig = IntegratedGradients(captum_model)
        ig_attr = ig.attribute(edge_mask.unsqueeze(0),
                               target=int(y[output_idx]),
                               additional_forward_args=(x, edge_index),
                               internal_batch_size=1)

    .. note::
        For an example of using a Captum attribution method within PyG, see
        `examples/captum_explainability.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        captum_explainability.py>`_.

    Args:
        model (torch.nn.Module): The model to be explained.
        mask_type (str, optional): Denotes the type of mask to be created with
            a Captum explainer. Valid inputs are :obj:`"edge"`, :obj:`"node"`,
            and :obj:`"node_and_edge"`:

            1. :obj:`"edge"`: The inputs to the forward function should be an
               edge mask tensor of shape :obj:`[1, num_edges]`, a regular
               :obj:`x` matrix and a regular :obj:`edge_index` matrix.

            2. :obj:`"node"`: The inputs to the forward function should be a
               node feature tensor of shape :obj:`[1, num_nodes, num_features]`
               and a regular :obj:`edge_index` matrix.

            3. :obj:`"node_and_edge"`: The inputs to the forward function
               should be a node feature tensor of shape
               :obj:`[1, num_nodes, num_features]`, an edge mask tensor of
               shape :obj:`[1, num_edges]` and a regular :obj:`edge_index`
               matrix.

            For all mask types, additional arguments can be passed to the
            forward function as long as the first arguments are set as
            described. (default: :obj:`"edge"`)
        output_idx (int, optional): Index of the output element (node or link
            index) to be explained. With :obj:`output_idx` set, the forward
            function will return the output of the model for the element at
            the index specified. (default: :obj:`None`)
    """
    return CaptumModel(model, mask_type, output_idx)



class Explainer(torch.nn.Module):
    r"""An abstract class for integrating explainability into Graph Neural
    Networks, *e.g.* :class:`~torch_geometric.nn.GNNExplainer` and
    :class:`~torch_geometric.nn.PGExplainer`.
    It also provides general visualization methods for graph attributions.

    Args:
        model (torch.nn.Module): The GNN module to explain.
        epochs (int, optional): The number of epochs to train.
            (default: :obj:`None`)
        lr (float, optional): The learning rate to apply.
            (default: :obj:`None`)
        num_hops (int, optional): The number of hops the :obj:`model` is
            aggregating information from.
            If set to :obj:`None`, will automatically try to detect this
            information based on the number of
            :class:`~torch_geometric.nn.conv.message_passing.MessagePassing`
            layers inside :obj:`model`. (default: :obj:`None`)
        return_type (str, optional): Denotes the type of output from
            :obj:`model`. Valid inputs are :obj:`"log_prob"` (the model
            returns the logarithm of probabilities), :obj:`"prob"` (the
            model returns probabilities), :obj:`"raw"` (the model returns raw
            scores) and :obj:`"regression"` (the model returns scalars).
            (default: :obj:`"log_prob"`)
        log (bool, optional): If set to :obj:`False`, will not log any learning
            progress. (default: :obj:`True`)
    """
    def __init__(self, model: torch.nn.Module, lr: Optional[float] = None,
                 epochs: Optional[int] = None, num_hops: Optional[int] = None,
                 return_type: str = 'log_prob', log: bool = False):
        super().__init__()
        assert return_type in ['log_prob', 'prob', 'raw', 'regression']

        self.model = model
        self.lr = lr
        self.epochs = epochs
        self.num_hops = num_hops or get_num_hops(self.model)
        self.return_type = return_type
        self.log = log

    def _flow(self) -> str:
        for module in self.model.modules():
            if isinstance(module, MessagePassing):
                return module.flow
        return 'source_to_target'

    def subgraph(self, node_idx: int, x: Tensor, edge_index: Tensor, **kwargs):
        r"""Returns the subgraph of the given node.

        Args:
            node_idx (int): The node to explain.
            x (Tensor): The node feature matrix.
            edge_index (LongTensor): The edge indices.
            **kwargs (optional): Additional arguments passed to the GNN module.
        :rtype: (Tensor, Tensor, LongTensor, LongTensor, LongTensor, dict)
        """
        num_nodes, num_edges = x.size(0), edge_index.size(0)
        subset, edge_index, mapping, edge_mask = k_hop_subgraph(
            node_idx, self.num_hops, edge_index, relabel_nodes=True,
            num_nodes=num_nodes, flow=self._flow())

        x = x[subset]
        kwargs_new = {}
        for key, value in kwargs.items():
            if torch.is_tensor(value) and value.size(0) == num_nodes:
                kwargs_new[key] = value[subset]
            elif torch.is_tensor(value) and value.size(0) == num_edges:
                kwargs_new[key] = value[edge_mask]
            kwargs_new[key] = value  # TODO: this is not in PGExplainer
        return x, edge_index, mapping, edge_mask, subset, kwargs_new


    def _to_log_prob(self, x):
        x = x.log_softmax(dim=-1) if self.return_type == 'raw' else x
        x = x.log() if self.return_type == 'prob' else x
        return x

    @torch.no_grad()
    def get_initial_prediction(self, x: Tensor, edge_index: Tensor,
                               batch: Optional[Tensor] = None, **kwargs):
        if batch is not None:
            out = self.model(x, edge_index, batch, **kwargs)
        else:
            out = self.model(x, edge_index, **kwargs)
        if self.return_type == 'regression':
            prediction = out
        else:
            log_logits = self._to_log_prob(out)
            prediction = log_logits.argmax(dim=-1)
        return prediction


    def get_loss(self, out: Tensor, prediction: Tensor,
                 node_idx: Optional[int] = None, **kwargs):
        if self.return_type == 'regression':
            loss = self._loss(out, prediction, node_idx, **kwargs)
        else:
            log_logits = self._to_log_prob(out)
            loss = self._loss(log_logits, prediction, node_idx, **kwargs)
        return loss


    def visualize_subgraph(self, node_idx: Optional[int], edge_index: Tensor,
                           edge_mask: Tensor, y: Optional[Tensor] = None,
                           threshold: Optional[int] = None,
                           edge_y: Optional[Tensor] = None,
                           node_alpha: Optional[Tensor] = None, seed: int = 10,
                           **kwargs):
        r"""Visualizes the subgraph given an edge mask :attr:`edge_mask`.

        Args:
            node_idx (int): The node id to explain.
                Set to :obj:`None` to explain a graph.
            edge_index (LongTensor): The edge indices.
            edge_mask (Tensor): The edge mask.
            y (Tensor, optional): The ground-truth node-prediction labels used
                as node colorings. All nodes will have the same color
                if :attr:`node_idx` is :obj:`-1`.(default: :obj:`None`).
            threshold (float, optional): Sets a threshold for visualizing
                important edges. If set to :obj:`None`, will visualize all
                edges with transparancy indicating the importance of edges.
                (default: :obj:`None`)
            edge_y (Tensor, optional): The edge labels used as edge colorings.
            node_alpha (Tensor, optional): Tensor of floats (0 - 1) indicating
                transparency of each node.
            seed (int, optional): Random seed of the :obj:`networkx` node
                placement algorithm. (default: :obj:`10`)
            **kwargs (optional): Additional arguments passed to
                :func:`nx.draw`.

        :rtype: :class:`matplotlib.axes.Axes`, :class:`networkx.DiGraph`
        """
        import matplotlib.pyplot as plt
        import networkx as nx

        assert edge_mask.size(0) == edge_index.size(1)

        if node_idx is None or node_idx < 0:
            hard_edge_mask = torch.BoolTensor([True] * edge_index.size(1),
                                              device=edge_mask.device)
            subset = torch.arange(edge_index.max().item() + 1,
                                  device=edge_index.device)
            y = None

        else:
            # Only operate on a k-hop subgraph around `node_idx`.
            subset, edge_index, _, hard_edge_mask = k_hop_subgraph(
                node_idx, self.num_hops, edge_index, relabel_nodes=True,
                num_nodes=None, flow=self._flow())

        edge_mask = edge_mask[hard_edge_mask]

        if threshold is not None:
            edge_mask = (edge_mask >= threshold).to(torch.float)

        if y is None:
            y = torch.zeros(edge_index.max().item() + 1,
                            device=edge_index.device)
        else:
            y = y[subset].to(torch.float) / y.max().item()

        if edge_y is None:
            edge_color = ['black'] * edge_index.size(1)
        else:
            colors = list(plt.rcParams['axes.prop_cycle'])
            edge_color = [
                colors[i % len(colors)]['color']
                for i in edge_y[hard_edge_mask]
            ]

        data = Data(edge_index=edge_index, att=edge_mask,
                    edge_color=edge_color, y=y, num_nodes=y.size(0)).to('cpu')
        G = to_networkx(data, node_attrs=['y'],
                        edge_attrs=['att', 'edge_color'])
        mapping = {k: i for k, i in enumerate(subset.tolist())}
        G = nx.relabel_nodes(G, mapping)

        node_args = set(signature(nx.draw_networkx_nodes).parameters.keys())
        node_kwargs = {k: v for k, v in kwargs.items() if k in node_args}
        node_kwargs['node_size'] = kwargs.get('node_size') or 800
        node_kwargs['cmap'] = kwargs.get('cmap') or 'cool'

        label_args = set(signature(nx.draw_networkx_labels).parameters.keys())
        label_kwargs = {k: v for k, v in kwargs.items() if k in label_args}
        label_kwargs['font_size'] = kwargs.get('font_size') or 10

        pos = nx.spring_layout(G, seed=seed)
        ax = plt.gca()
        for source, target, data in G.edges(data=True):
            ax.annotate(
                '', xy=pos[target], xycoords='data', xytext=pos[source],
                textcoords='data', arrowprops=dict(
                    arrowstyle="->",
                    alpha=max(data['att'], 0.1),
                    color=data['edge_color'],
                    shrinkA=sqrt(node_kwargs['node_size']) / 2.0,
                    shrinkB=sqrt(node_kwargs['node_size']) / 2.0,
                    connectionstyle="arc3,rad=0.1",
                ))

        if node_alpha is None:
            nx.draw_networkx_nodes(G, pos, node_color=y.tolist(),
                                   **node_kwargs)
        else:
            node_alpha_subset = node_alpha[subset]
            assert ((node_alpha_subset >= 0) & (node_alpha_subset <= 1)).all()
            nx.draw_networkx_nodes(G, pos, alpha=node_alpha_subset.tolist(),
                                   node_color=y.tolist(), **node_kwargs)

        nx.draw_networkx_labels(G, pos, **label_kwargs)

        return ax, G

In [6]:
def get_num_hops(model: torch.nn.Module) -> int:
    r"""Returns the number of hops the model is aggregating information
    from."""
    from torch_geometric.nn.conv import MessagePassing
    num_hops = 0
    for module in model.modules():
        if isinstance(module, MessagePassing):
            num_hops += 1
    return num_hops

In [39]:
surrogate_model = model.architectures[0].repackage_into_one_sequential()

output_idx = 2

captum_model = to_captum(surrogate_model, mask_type='node_and_edge',
                         output_idx=output_idx)

ig = IntegratedGradients(captum_model)
ig_attr_node, ig_attr_edge = ig.attribute(
    (latent_features.unsqueeze(0), edge_mask.unsqueeze(0)), target=target,
    additional_forward_args=(data.edge_index), internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_node = ig_attr_node.squeeze(0).abs().sum(dim=1)
ig_attr_node /= ig_attr_node.max()
ig_attr_edge = ig_attr_edge.squeeze(0).abs()
ig_attr_edge /= ig_attr_edge.max()

fig, ax = plt.subplots(figsize=(15,15))
# Visualize absolute values of attributions:
explainer = Explainer(surrogate_model)
ax, G = explainer.visualize_subgraph(output_idx, data.edge_index, ig_attr_edge,
                                     node_alpha=ig_attr_node)

fig.tight_layout()
plt.show()


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [13]:
latent_fetures = captum_model.model.architectures[0].pre_mp(dataset.data.x.float())
mp_layers = model.get_mp_layers()
captum_model.model.architectures[0].mp.forward(latent_fetures, dataset.data.edge_index, edge_mask)

tensor([[-0.2894, -1.7373,  0.1016,  ...,  0.6048, -0.1413,  0.5270],
        [-0.4566, -0.1034,  0.1626,  ...,  0.3115,  0.0902, -0.3112],
        [ 0.0419,  1.0243,  1.0386,  ..., -0.1690,  0.2115,  1.0922],
        ...,
        [ 0.1602,  1.7448,  0.4021,  ...,  2.0865, -0.2629,  0.1893],
        [-0.2524,  0.1203,  0.1267,  ...,  0.5496,  0.0370, -0.1255],
        [ 0.1806, -0.1830,  0.0414,  ..., -0.3247, -0.2045,  0.4281]],
       grad_fn=<TBackward>)

In [14]:
mp_layers = model.get_mp_layers()

In [22]:
mp_layers[0](latent_fetures, dataset.data.edge_index, edge_mask*1)

tensor([ 0.3666, -0.2685,  0.3210, -0.2911,  0.2778, -0.2067,  0.3706,  0.1526,
         0.0399, -0.0883, -0.0877, -0.3720,  0.2127, -0.5178, -0.1023,  0.0809,
         0.1777, -0.0036,  0.1919,  0.0357,  0.2230, -0.0901, -0.3024, -0.2726,
        -0.0447, -0.2677,  0.5260, -0.7340, -0.1083,  0.4826,  0.1412,  0.2478,
        -0.2257,  0.1827,  0.2730, -0.5653,  0.4987, -0.0307,  0.2041,  0.0942,
        -0.0540,  0.1655, -0.1854, -0.0147, -0.1483,  0.1138, -0.7409, -0.0686,
        -0.3206, -0.0717], grad_fn=<SliceBackward>)

In [25]:
captum_model.model(dataset.data.x.float(), dataset.data.edge_index, edge_mask)

tensor([[ 0.5744],
        [-0.1350],
        [ 0.7640],
        ...,
        [ 0.2806],
        [ 0.1062],
        [ 0.1690]], grad_fn=<AddmmBackward>)

In [43]:
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN()

In [44]:
# Edge Explainability
device = "cpu"
output_idx = 14
target = int(dataset.data.y[output_idx])

captum_model = to_captum(model, mask_type='edge', output_idx=output_idx)
edge_mask = torch.ones(dataset.data.num_edges, requires_grad=True, device=device)

ig = IntegratedGradients(captum_model)
ig_attr_edge = ig.attribute(edge_mask.unsqueeze(0), target=target,
                            additional_forward_args=(dataset.data.x.float(), dataset.data.edge_index),
                            internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_edge = ig_attr_edge.squeeze(0).abs()
ig_attr_edge /= ig_attr_edge.max()


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [37]:
data = dataset.data
model = model.float()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    log_logits = model(data.x.float(), data.edge_index)
    loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask].long())
    loss.backward()
    optimizer.step()

In [46]:

dataset = 'Cora'
path = osp.join('..', 'data', 'Planetoid')
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    log_logits = model(data.x, data.edge_index)
    loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

output_idx = 10
target = int(data.y[output_idx])

# Edge explainability
# ===================

# Captum assumes that for all given input tensors, dimension 0 is
# equal to the number of samples. Therefore, we use unsqueeze(0).
captum_model = to_captum(model, mask_type='edge', output_idx=output_idx)
edge_mask = torch.ones(data.num_edges, requires_grad=True, device=device)

ig = IntegratedGradients(captum_model)
ig_attr_edge = ig.attribute(edge_mask.unsqueeze(0), target=target,
                            additional_forward_args=(data.x, data.edge_index),
                            internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_edge = ig_attr_edge.squeeze(0).abs()
ig_attr_edge /= ig_attr_edge.max()

# Visualize absolute values of attributions:
explainer = Explainer(model)
ax, G = explainer.visualize_subgraph(output_idx, data.edge_index, ig_attr_edge)
plt.show()

Processing...
Done!


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [75]:
explainer = Explainer(model)
output_idx = 22
target = int(dataset.data.y[output_idx])
captum_model = to_captum(model, mask_type='node_and_edge',
                         output_idx=output_idx)

In [44]:
output_idx = 15
target = int(dataset.data.y[output_idx])
captum_model = to_captum(model, mask_type='node_and_edge',
                         output_idx=output_idx)

In [38]:

torch.set_default_tensor_type(torch.FloatTensor)

ig = IntegratedGradients(captum_model)
edge_mask = torch.ones(dataset.data.num_edges, requires_grad=True)

ig_attr_node, ig_attr_edge = ig.attribute(
    dataset.data.x.float().unsqueeze(0), target=0,
    additional_forward_args=dataset.data.edge_index, internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_node = ig_attr_node.squeeze(0).abs().sum(dim=1)
ig_attr_node /= ig_attr_node.max()
ig_attr_edge = ig_attr_edge.squeeze(0).abs()
ig_attr_edge /= ig_attr_edge.max()

AssertionError: Expects at least x and edge_index as args.

In [21]:
model(dataset.data.x.float(), dataset.data.edge_index)


TypeError: forward() takes 3 positional arguments but 4 were given

In [63]:
data = dataset.data

In [82]:

# Edge explainability
# ===================

# Captum assumes that for all given input tensors, dimension 0 is
# equal to the number of samples. Therefore, we use unsqueeze(0).
"""
for output_idx in range(data.y.shape[0]):
    target = int(dataset.data.y[output_idx])
    captum_model = to_captum(model, mask_type='edge', output_idx=output_idx)
    edge_mask = torch.ones(data.num_edges, requires_grad=True, device="cpu").float()

    ig = IntegratedGradients(captum_model)
    try:
        ig_attr_edge = ig.attribute(edge_mask.unsqueeze(0).float(), target=target,
                                    additional_forward_args=(data.x.float(), data.edge_index),
                                    internal_batch_size=1)
    except RuntimeError:
        continue
    except IndexError:
        continue
    if output_idx % 200 == 0:
        print("Tried {} nodes".format(output_idx))
    break
"""
output_idx=22
target = int(dataset.data.y[output_idx])
captum_model = to_captum(model, mask_type='edge', output_idx=output_idx)
edge_mask = torch.ones(data.num_edges, requires_grad=True, device="cpu").float()

ig = IntegratedGradients(captum_model)
ig_attr_edge = ig.attribute(edge_mask.float().unsqueeze(0), target=1,
                            additional_forward_args=(data.x.float(), data.edge_index),
                            internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_edge = ig_attr_edge.squeeze(0).abs()
ig_attr_edge /= ig_attr_edge.max()

# Visualize absolute values of attributions:
explainer = Explainer(model)
ax, G = explainer.visualize_subgraph(output_idx, data.edge_index, ig_attr_edge)
plt.show()

/home/icb/florin.ratajczak/anaconda3/envs/compat/lib/python3.7/site-packages/captum/attr/_utils/batching.py:48: UserWarning: Internal batch size cannot be less than the number of input examples. Defaulting to internal batch size of 14882 equal to the number of examples.
  % num_examples


AssertionError: Dimension 0 of input should be 1

In [65]:
captum_model = to_captum(model, mask_type='node', output_idx=output_idx)

ig = IntegratedGradients(captum_model)
ig_attr_node = ig.attribute(data.x.unsqueeze(0).float(), target=target,
                            additional_forward_args=(data.edge_index),
                            internal_batch_size=1)

# Scale attributions to [0, 1]:
ig_attr_node = ig_attr_node.squeeze(0).abs().sum(dim=1)
ig_attr_node /= ig_attr_node.max()

# Visualize absolute values of attributions:
ax, G = explainer.visualize_subgraph(output_idx, data.edge_index, ig_attr_edge,
                                     node_alpha=ig_attr_node)
plt.show()


NameError: name 'ig_attr_edge' is not defined

In [72]:
data.y.shape[0]

17320